# Apriomics Visualization Demo

This notebook demonstrates the visualization capabilities of the apriomics package for Bayesian metabolomics priors.

## Features Demonstrated:
1. **Markov Field Priors** from metabolic reaction networks
2. **Bayesian Prior Distributions** with uncertainty quantification
3. **Network Analysis** and statistics
4. **Comprehensive Reporting**

In [ ]:
# Setup and imports
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from dataclasses import dataclass
from typing import List
import networkx

# Add src to path for imports
sys.path.insert(0, str(Path.cwd().parent / "src"))

# Set matplotlib backend for Jupyter
%matplotlib inline
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 10

In [ ]:
# Import apriomics components
from apriomics.build_signed_edges import process_reactions
from apriomics.visualization import MarkovFieldVisualizer, LLMPriorVisualizer, analyze_priors

@dataclass
class MetaboliteScore:
    """Simple structure for metabolite scoring with Bayesian priors."""
    metabolite: str
    score: float
    direction: str
    rationale: str
    expected_log2fc: float = 0.0
    prior_sd: float = 0.5
    magnitude: str = "moderate"
    confidence: str = "moderate"

✅ Imports successful!


## 1. Create Example Data

We'll create example signed edges from metabolic reactions and Bayesian prior scores for metabolites.

In [4]:
def create_example_signed_edges():
    """Create example signed edges from metabolic reactions."""
    print("Creating signed metabolic network...")
    
    # Create mock signed edges
    metabolites = [
        "C00031",  # D-Glucose
        "C00092",  # D-Glucose 6-phosphate
        "C00085",  # D-Fructose 6-phosphate
        "C00354",  # D-Fructose 1,6-bisphosphate
        "C00111",  # Glycerone phosphate
        "C00118",  # D-Glyceraldehyde 3-phosphate
        "C00022",  # Pyruvate
        "C00036",  # Oxaloacetate
        "C00158",  # Citrate
        "C00311",  # Isocitrate
    ]
    
    signed_edges = {}
    
    # Add substrate-product relationships (-1)
    substrate_product_pairs = [
        ("C00031", "C00092"),  # Glucose -> G6P
        ("C00092", "C00085"),  # G6P -> F6P
        ("C00085", "C00354"),  # F6P -> F1,6BP
        ("C00354", "C00111"),  # F1,6BP -> DHAP
        ("C00354", "C00118"),  # F1,6BP -> G3P
        ("C00118", "C00022"),  # G3P -> Pyruvate
        ("C00158", "C00311"),  # Citrate -> Isocitrate
        ("C00022", "C00036"),  # Pyruvate -> Oxaloacetate
        ("C00036", "C00158"),  # Oxaloacetate -> Citrate
    ]
    
    for sub, prod in substrate_product_pairs:
        signed_edges[(sub, prod)] = -1
        signed_edges[(prod, sub)] = -1  # Reversible
    
    # Add same-side relationships (+1)
    same_side_pairs = [
        ("C00111", "C00118"),  # DHAP and G3P (both products of F1,6BP)
        ("C00092", "C00031"),  # G6P and Glucose (related)
        ("C00158", "C00036"),  # Citrate and Oxaloacetate (TCA cycle)
    ]
    
    for met1, met2 in same_side_pairs:
        signed_edges[(met1, met2)] = 1
        signed_edges[(met2, met1)] = 1
    
    return signed_edges

# Create the signed edges
signed_edges = create_example_signed_edges()
print(f"Generated {len(signed_edges)} signed edges")

Creating signed metabolic network...
Generated 20 signed edges


In [5]:
def create_example_bayesian_scores():
    """Create example Bayesian metabolite prior scores."""
    print("Creating Bayesian metabolite prior scores...")
    
    # Example metabolites with realistic prior scoring for diabetes study
    example_metabolites = [
        ("Glucose", "increase", "large", "high", "Primary substrate elevated in diabetes"),
        ("Lactate", "increase", "moderate", "high", "Elevated due to anaerobic metabolism"),
        ("Citrate", "decrease", "small", "moderate", "Reduced TCA cycle activity"),
        ("Alanine", "increase", "small", "moderate", "Increased protein catabolism"),
        ("Glutamine", "decrease", "moderate", "high", "Consumed for energy production"),
        ("Pyruvate", "unclear", "moderate", "low", "Variable depending on metabolic state"),
        ("Acetate", "minimal", "minimal", "high", "Generally stable across conditions"),
        ("Creatine", "increase", "small", "moderate", "Muscle-related changes"),
        ("Succinate", "increase", "moderate", "moderate", "Mitochondrial dysfunction marker"),
        ("Fumarate", "increase", "small", "low", "Associated with metabolic stress"),
        ("Glycine", "decrease", "small", "moderate", "Reduced in metabolic dysfunction"),
        ("Serine", "increase", "small", "high", "Elevated in diabetes pathways"),
    ]
    
    bayesian_scores = []
    
    for metabolite, direction, magnitude, confidence, rationale in example_metabolites:
        # Calculate expected log2fc and prior_sd based on direction and magnitude
        direction_mapping = {
            'increase': 1,
            'decrease': -1,
            'minimal': 0,
            'unclear': 0
        }
        
        magnitude_mapping = {
            'minimal': 0.3,
            'small': 0.6,
            'moderate': 1.0,
            'large': 1.6
        }
        
        confidence_mapping = {
            'high': 0.8,
            'moderate': 1.0,
            'low': 1.4
        }
        
        expected_log2fc = direction_mapping[direction] * magnitude_mapping[magnitude]
        prior_sd = 0.3 * magnitude_mapping[magnitude] * confidence_mapping[confidence]
        
        # Calculate importance score (0-1) based on abs(log2fc)
        importance_score = min(abs(expected_log2fc) / 2.0, 1.0)
        
        score = MetaboliteScore(
            metabolite=metabolite,
            score=importance_score,
            direction=direction,
            rationale=rationale,
            expected_log2fc=expected_log2fc,
            prior_sd=prior_sd,
            magnitude=magnitude,
            confidence=confidence
        )
        
        bayesian_scores.append(score)
    
    return bayesian_scores

# Create the Bayesian scores
bayesian_scores = create_example_bayesian_scores()
print(f"Generated {len(bayesian_scores)} Bayesian prior scores")

Creating Bayesian metabolite prior scores...
Generated 12 Bayesian prior scores


## 2. Initialize Visualizers

Create visualizer instances for both Markov field and Bayesian priors.

In [6]:
# Create visualizers
print("Creating visualizers...")
markov_viz = MarkovFieldVisualizer(signed_edges)
bayesian_viz = LLMPriorVisualizer(bayesian_scores)

# Display statistics
print("\n📊 Network Statistics:")
network_stats = markov_viz.get_network_statistics()
for key, value in network_stats.items():
    print(f"  {key}: {value}")

print("\n📊 Bayesian Prior Statistics:")
bayesian_stats = bayesian_viz.get_summary_statistics()
for key, value in bayesian_stats.items():
    print(f"  {key}: {value}")

Creating visualizers...

📊 Network Statistics:
  error: NetworkX required for network statistics

📊 Bayesian Prior Statistics:
  num_metabolites: 12
  mean_importance: 0.3416666666666666
  std_importance: 0.2193309385519075
  direction_distribution: {'increase': 7, 'decrease': 3, 'unclear': 1, 'minimal': 1}
  confidence_distribution: {'high': 5, 'moderate': 5, 'low': 2}
  magnitude_distribution: {'small': 6, 'moderate': 4, 'large': 1, 'minimal': 1}
  mean_expected_log2fc: 0.3166666666666667
  mean_prior_sd: 0.23099999999999998


## 3. Markov Field Visualizations

Visualize the signed metabolic network and its properties.

In [7]:
# Plot the signed metabolic network
print("🔗 Plotting signed metabolic network...")
try:
    fig1 = markov_viz.plot_network(figsize=(12, 8), node_size=500)
    plt.tight_layout()
    plt.show()
    print("✅ Network plot generated successfully")
except Exception as e:
    print(f"❌ Network plot failed: {e}")
    print("💡 This requires NetworkX. Try: uv add networkx")

🔗 Plotting signed metabolic network...
❌ Network plot failed: NetworkX required for network visualization
💡 This requires NetworkX. Try: uv add networkx


In [ ]:
# Plot the Laplacian matrix heatmap
print("🔥 Plotting Laplacian matrix heatmap...")
try:
    fig2 = markov_viz.plot_laplacian_heatmap(figsize=(10, 8))
    plt.tight_layout()
    plt.show()
    print("✅ Laplacian heatmap generated successfully")
except Exception as e:
    print(f"❌ Laplacian heatmap failed: {e}")

In [ ]:
# Plot degree distribution
print("📈 Plotting degree distribution...")
try:
    fig3 = markov_viz.plot_degree_distribution(figsize=(12, 6))
    plt.tight_layout()
    plt.show()
    print("✅ Degree distribution generated successfully")
except Exception as e:
    print(f"❌ Degree distribution failed: {e}")
    print("💡 This requires NetworkX. Try: uv add networkx")

## 4. Bayesian Prior Visualizations

Visualize the Bayesian metabolite priors and their properties.

In [ ]:
# Plot importance scores
print("⭐ Plotting Bayesian importance scores...")
try:
    fig4 = bayesian_viz.plot_importance_scores(top_n=12, figsize=(12, 8))
    plt.tight_layout()
    plt.show()
    print("✅ Importance scores generated successfully")
except Exception as e:
    print(f"❌ Importance scores failed: {e}")

In [ ]:
# Plot prior distributions
print("📊 Plotting prior distributions...")
try:
    fig5 = bayesian_viz.plot_prior_distributions(figsize=(15, 10))
    plt.tight_layout()
    plt.show()
    print("✅ Prior distributions generated successfully")
except Exception as e:
    print(f"❌ Prior distributions failed: {e}")

In [ ]:
# Plot confidence analysis
print("🎯 Plotting confidence analysis...")
try:
    fig6 = bayesian_viz.plot_confidence_analysis(figsize=(12, 6))
    plt.tight_layout()
    plt.show()
    print("✅ Confidence analysis generated successfully")
except Exception as e:
    print(f"❌ Confidence analysis failed: {e}")

In [ ]:
# Plot direction analysis
print("🧭 Plotting direction analysis...")
try:
    fig7 = bayesian_viz.plot_direction_analysis(figsize=(12, 8))
    plt.tight_layout()
    plt.show()
    print("✅ Direction analysis generated successfully")
except Exception as e:
    print(f"❌ Direction analysis failed: {e}")

## 5. Interactive Exploration

Create an interactive widget to explore individual metabolites (if ipywidgets is available).

In [ ]:
# Try to create interactive explorer
print("🔍 Creating interactive explorer...")
try:
    interactive_widget = bayesian_viz.create_interactive_explorer()
    if interactive_widget:
        display(interactive_widget)
        print("✅ Interactive explorer created!")
    else:
        print("❌ Interactive explorer not available")
        print("💡 Install ipywidgets: uv add ipywidgets")
except Exception as e:
    print(f"❌ Interactive explorer failed: {e}")
    print("💡 Install ipywidgets: uv add ipywidgets")

## 6. Generate Comprehensive Report

Create a comprehensive analysis report with all visualizations saved to files.

In [ ]:
# Generate comprehensive report
print("📋 Generating comprehensive report...")
try:
    analyze_priors(signed_edges, bayesian_scores, output_dir="../output/visualization_report")
    print("✅ Comprehensive report generated successfully!")
    print("📁 Check the '../output/visualization_report/' directory for saved plots and statistics")
except Exception as e:
    print(f"❌ Comprehensive report failed: {e}")

## 7. Summary and Next Steps

The apriomics visualization suite provides comprehensive tools for analyzing metabolomics priors.

In [ ]:
print("🎉 APRIOMICS VISUALIZATION DEMO COMPLETED!")
print("=" * 50)
print("\n📈 What you've seen:")
print("  ✅ Signed metabolic networks (Markov field priors)")
print("  ✅ Bayesian prior distributions with uncertainty")
print("  ✅ Comprehensive statistical analysis")
print("  ✅ Interactive exploration capabilities")
print("  ✅ Automated report generation")

print("\n🚀 Next steps for your research:")
print("  1. Replace example data with your metabolomics data")
print("  2. Use chembridge for metabolite name standardization")
print("  3. Integrate with your Bayesian modeling pipeline")
print("  4. Customize visualizations for your specific needs")
print("  5. Share comprehensive reports with collaborators")

print("\n📚 Learn more:")
print("  - Check the CLAUDE.md file for project documentation")
print("  - Explore examples/ directory for more use cases")
print("  - Read the visualization.py module for advanced customization")